## <font color='darkblue'>Preface</font>
([article source](https://machinelearningmastery.com/implementing-the-transformer-encoder-from-scratch-in-tensorflow-and-keras)) Having seen how to implement the [**scaled dot-product attention**](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras), and integrate it within the [**multi-head attention**](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras) of the Transformer model, we may progress one step further towards implementing a complete Transformer model by implementing its encoder. Our end goal remains the application of the complete model to Natural Language Processing (<font color='brown'>NLP</font>).

In this tutorial, you will discover how to implement the Transformer encoder from scratch in TensorFlow and Keras. After completing this tutorial, you will know:
* The layers that form part of the Transformer encoder.
* How to implement the Transformer encoder from scratch.  

### <font color='darkgreen'>Tutorial Overview</font>
This tutorial is divided into three parts; they are:
* [**Recap of the Transformer Architecture**](#sect1)
    * [The Transformer Encoder](#sect1_1)
* [**Implementing the Transformer Encoder From Scratch**](#sect2)
    * [The Fully Connected Feed-Forward Neural Network and Layer Normalization](#sect2_1)
    * [The Encoder Layer](#sect2_2)
    * [The Transformer Encoder](#sect2_3)
* [**Testing Out the Code**](#sect3)

### <font color='darkgreen'>Prerequisites</font>
For this tutorial, we assume that you are already familiar with:
* [The Transformer model](https://machinelearningmastery.com/the-transformer-model/)
* [The scaled dot-product attention](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras)
* [The multi-head attention](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras)
* [The Transformer positional encoding](https://machinelearningmastery.com/the-transformer-positional-encoding-layer-in-keras-part-2/)

<a id='sect1'></a>
## <font color='darkblue'>Recap of the Transformer Architecture</font>
[Recall](https://machinelearningmastery.com/the-transformer-model/) having seen that the Transformer architecture follows an encoder-decoder structure: the encoder, on the left-hand side, is tasked with mapping an input sequence to a sequence of continuous representations; the decoder, on the right-hand side, receives the output of the encoder together with the decoder output at the previous time step, to generate an output sequence.
![Transformer architecture](images/1.PNG)

<br/>

In generating an output sequence, the Transformer does not rely on recurrence and convolutions.

We had seen that the decoder part of the Transformer shares many similarities in its architecture with the encoder. <b>In this tutorial, we will be focusing on the components that form part of the Transformer encoder</b>.

<a id='sect1_1'></a>
### <font color='darkgreen'>The Transformer Encoder</font>
The Transformer encoder consists of a stack of  identical layers, where each layer further consists of two main sub-layers:
* **The first sub-layer comprises a multi-head attention mechanism** that receives the queries, keys and values as inputs.
* **A second sub-layer that comprises a fully-connected feed-forward network**. 

<br/>

Following each of these two sub-layers is layer normalisation, into which the sub-layer input (<font color='brown'>through a residual connection</font>) and output are fed. The output of each layer normalization step is the following:
> LayerNorm(Sublayer Input + Sublayer Output)

<br/>

In order to facilitate such an operation, which involves an addition between the sublayer input and output, Vaswani et al. designed all sub-layers and embedding layers in the model to produce outputs of dimension, $d_{model} = 512$

[Recall](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras) as well the queries, keys and values as the inputs to the Transformer encoder.

Here, the queries, keys and values carry the same input sequence after this has been embedded and augmented by positional information, where the queries and keys are of dimensionality, $d_{k}$, whereas the dimensionality of the values is $d_{v}$.

Furthermore, Vaswani et al. also introduce regularization into the model by applying dropout to the output of each sub-layer (<font color='brown'>before the layer normalization step</font>), as well as to the positional encodings before these are fed into the encoder. 

Let’s now see how to implement the Transformer encoder from scratch in TensorFlow and Keras.

<a id='sect2'></a>
## <font color='darkblue'>Implementing the Transformer Encoder From Scratch</font>

### <font color='darkgreen'>The Fully Connected Feed-Forward Neural Network and Layer Normalization</font>
<b><font size='3ptx'>We shall begin by creating classes for the Feed Forward and Add & Norm layers that are shown in the diagram above.</font></b>

Vaswani et al. tell us that the fully connected feed-forward network consists of two linear transformations with a ReLU activation in between. The first linear transformation produces an output of dimensionality, $d_{ff} = 2048$, while the second linear transformation produces an output of dimensionality, $d_{model} = 512$.

For this purpose, let’s first create the class, <b><font color='blue'>FeedForward</font></b> that inherits form the [**Layer**](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) base class in Keras, and initialize the dense layers and the ReLU activation:
```python
class FeedForward(Layer):
    def __init__(self, d_ff, d_model, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fully_connected1 = Dense(d_ff)  # First fully connected layer
        self.fully_connected2 = Dense(d_model)  # Second fully connected layer
        self.activation = ReLU()  # ReLU activation layer
        ...
```

<br/>

We will add to it the class method, <font color='blue'>call()</font>, that receives an input and passes it through the two fully connected layers with ReLU activation, returning an output of dimensionality equal to 512:
```python
...
def call(self, x):
    # The input is passed into the two fully-connected layers, with a ReLU in between
    x_fc1 = self.fully_connected1(x)

    return self.fully_connected2(self.activation(x_fc1))
```

<br/>

The next step is to create another class, <font color='blue'><b>AddNormalization</b></font>, that also inherits form the [**Layer**](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) base class in Keras, and initialize a Layer normalization layer:
```python
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()  # Layer normalization layer
        ...
```

<br/>

In it, we will include the following class method that <b>sums its sub-layer’s input and output, which it receives as inputs, and applies layer normalization to the result</b>:
```python
...
def call(self, x, sublayer_x):
    # The sublayer input and output need to be of the same shape to be summed
    add = x + sublayer_x

    # Apply layer normalization to the sum
    return self.layer_norm(add)
```

<a id='sect2_2'></a>
### <font color='darkgreen'>The Encoder Layer</font>
<b><font size='3ptx'>Next, we will implement the encoder layer, which the Transformer encoder will replicate identically `N` times. </font></b>

For this purpose, let’s create the class, <font color='blue'><b>EncoderLayer</b></font>, and initialize all of the sub-layers that it consists of:
```python
class EncoderLayer(Layer):
    def __init__(self, h, d_k, d_v, d_model, d_ff, rate, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.multihead_attention = MultiHeadAttention(h, d_k, d_v, d_model)
        self.dropout1 = Dropout(rate)
        self.add_norm1 = AddNormalization()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout2 = Dropout(rate)
        self.add_norm2 = AddNormalization()
        ...
```

<br/>

Here you may notice that we have initialized instances of the <b><font color='blue'>FeedForward</font></b> and <b><font color='blue'>AddNormalization</font></b> classes, which we have just created in the previous section, and assigned their output to the respective variables, `feed_forward` and `add_norm` (<font color='brown'>1 and 2</font>). The [**Dropout**](https://keras.io/api/layers/regularization_layers/dropout/) layer is self-explanatory, where rate defines the frequency at which the input units are set to 0. We had created the <b><font color='blue'>MultiHeadAttention</font></b> class in a [previous tutorial](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras), and if you had saved the code into a separate Python script, then do not forget to import it. I saved mine in a Python script named, `multihead_attention.py`, and for this reason I need to include the line of code.

Let’s now proceed to create the class method, <font color='blue'>call()</font>, that implements all of the encoder sub-layers:
```python
...
def call(self, x, padding_mask, training):
    # Multi-head attention layer
    multihead_output = self.multihead_attention(x, x, x, padding_mask)
    # Expected output shape = (batch_size, sequence_length, d_model)

    # Add in a dropout layer
    multihead_output = self.dropout1(multihead_output, training=training)

    # Followed by an Add & Norm layer
    addnorm_output = self.add_norm1(x, multihead_output)
    # Expected output shape = (batch_size, sequence_length, d_model)

    # Followed by a fully connected layer
    feedforward_output = self.feed_forward(addnorm_output)
    # Expected output shape = (batch_size, sequence_length, d_model)

    # Add in another dropout layer
    feedforward_output = self.dropout2(feedforward_output, training=training)

    # Followed by another Add & Norm layer
    return self.add_norm2(addnorm_output, feedforward_output)
```

<br/>

In addition to the input data, the <font color='blue'>call()</font> method can also receive a `padding mask`. As a brief reminder of what we had said in a [previous tutorial](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras), <b>the `padding mask` is necessary to suppress the zero padding in the input sequence from being processed along with the actual input values</b>. 

The same class method can receive a <font color='violet'>training</font> flag which, when set to True, will only apply the [**Dropout**](https://keras.io/api/layers/regularization_layers/dropout/) layers during training.

<a id='sect2_3'></a>
### <font color='darkgreen'>The Transformer Encoder</font>
The last step is to create a class for the Transformer encoder, which we shall be naming <b><font color='blue'>Encoder</font></b>:
```python
class Encoder(Layer):
    def __init__(self, vocab_size, sequence_length, h, d_k, d_v, d_model, d_ff, n, rate, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.pos_encoding = PositionEmbeddingFixedWeights(sequence_length, vocab_size, d_model)
        self.dropout = Dropout(rate)
        self.encoder_layer = [EncoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(n)]
        ...
```

<br/>

The Transformer encoder receives an input sequence after this would have undergone a process of word embedding and positional encoding. In order to compute the positional encoding, we will make use of the <b><font color='blue'>PositionEmbeddingFixedWeights</font></b> class described by Mehreen Saeed in [this tutorial](https://machinelearningmastery.com/the-transformer-positional-encoding-layer-in-keras-part-2/). 

As we have similarly done in the previous sections, here we will also create a class method, <font color='blue'>call()</font>, that applies word embedding and positional encoding to the input sequence, and feeds the result to `N` encoder layers:
```python
...
def call(self, input_sentence, padding_mask, training):
    # Generate the positional encoding
    pos_encoding_output = self.pos_encoding(input_sentence)
    # Expected output shape = (batch_size, sequence_length, d_model)

    # Add in a dropout layer
    x = self.dropout(pos_encoding_output, training=training)

    # Pass on the positional encoded values to each encoder layer
    for i, layer in enumerate(self.encoder_layer):
        x = layer(x, padding_mask, training)

    return x
```

<br/>

The code listing for the full Transformer encoder is the following:

In [2]:
from tensorflow.keras.layers import LayerNormalization, Layer, Dense, ReLU, Dropout
from multihead_attention import MultiHeadAttention
from positional_encoding import PositionEmbeddingFixedWeights

# Implementing the Add & Norm Layer
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()  # Layer normalization layer

    def call(self, x, sublayer_x):
        # The sublayer input and output need to be of the same shape to be summed
        add = x + sublayer_x

        # Apply layer normalization to the sum
        return self.layer_norm(add)

      
# Implementing the Feed-Forward Layer
class FeedForward(Layer):
    def __init__(self, d_ff, d_model, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fully_connected1 = Dense(d_ff)  # First fully connected layer
        self.fully_connected2 = Dense(d_model)  # Second fully connected layer
        self.activation = ReLU()  # ReLU activation layer

    def call(self, x):
        # The input is passed into the two fully-connected layers, with a ReLU in between
        x_fc1 = self.fully_connected1(x)

        return self.fully_connected2(self.activation(x_fc1))

      
# Implementing the Encoder Layer
class EncoderLayer(Layer):
    def __init__(self, h, d_k, d_v, d_model, d_ff, rate, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.multihead_attention = MultiHeadAttention(h, d_k, d_v, d_model)
        self.dropout1 = Dropout(rate)
        self.add_norm1 = AddNormalization()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout2 = Dropout(rate)
        self.add_norm2 = AddNormalization()

    def call(self, x, padding_mask, training):
        # Multi-head attention layer
        multihead_output = self.multihead_attention(x, x, x, padding_mask)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in a dropout layer
        multihead_output = self.dropout1(multihead_output, training=training)

        # Followed by an Add & Norm layer
        addnorm_output = self.add_norm1(x, multihead_output)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Followed by a fully connected layer
        feedforward_output = self.feed_forward(addnorm_output)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in another dropout layer
        feedforward_output = self.dropout2(feedforward_output, training=training)

        # Followed by another Add & Norm layer
        return self.add_norm2(addnorm_output, feedforward_output)

# Implementing the Encoder
class Encoder(Layer):
    def __init__(self, vocab_size, sequence_length, h, d_k, d_v, d_model, d_ff, n, rate, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.pos_encoding = PositionEmbeddingFixedWeights(sequence_length, vocab_size, d_model)
        self.dropout = Dropout(rate)
        self.encoder_layer = [EncoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(n)]

    def call(self, input_sentence, padding_mask, training):
        # Generate the positional encoding
        pos_encoding_output = self.pos_encoding(input_sentence)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in a dropout layer
        x = self.dropout(pos_encoding_output, training=training)

        # Pass on the positional encoded values to each encoder layer
        for i, layer in enumerate(self.encoder_layer):
            x = layer(x, padding_mask, training)

        return x

<a id='sect3'></a>
## <font color='darkblue'>Testing Out the Code</font>
We will be working with the parameter values specified in the paper, [Attention Is All You Need](https://arxiv.org/abs/1706.03762), by Vaswani et al. (2017):
```python
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 2048  # Dimensionality of the inner fully connected layer
d_model = 512  # Dimensionality of the model sub-layers' outputs
n = 6  # Number of layers in the encoder stack

batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers
...
```

<br/>

As for the input sequence we will be working with dummy data for the time being until we arrive to the stage of [training the complete Transformer model](https://machinelearningmastery.com/training-the-transformer-model) in a separate tutorial, at which point we will be using actual sentences:
```python
...
enc_vocab_size = 20 # Vocabulary size for the encoder
input_seq_length = 5  # Maximum length of the input sequence

input_seq = random.random((batch_size, input_seq_length))
...
```

<br/>

Next, we will create a new instance of the <b><font color='blue'>Encoder</font></b> class, assigning its output to the `encoder` variable, and subsequently feeding in the input arguments and printing the result. We will be setting the padding mask argument to `None` for the time being, but we shall return to this when we implement the complete Transformer model:
```python
...
encoder = Encoder(enc_vocab_size, input_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)
print(encoder(input_seq, None, True))
```

<br/>

Tying everything together produces the following code listing:

In [3]:
from numpy import random

enc_vocab_size = 20 # Vocabulary size for the encoder
input_seq_length = 5  # Maximum length of the input sequence
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 2048  # Dimensionality of the inner fully connected layer
d_model = 512  # Dimensionality of the model sub-layers' outputs
n = 6  # Number of layers in the encoder stack

batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers

input_seq = random.random((batch_size, input_seq_length))

encoder = Encoder(enc_vocab_size, input_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)
print(encoder(input_seq, None, True))

2022-10-15 19:39:48.589721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-15 19:39:48.590485: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-15 19:39:48.590616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2022-10-15 19:39:48.593622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.Tensor(
[[[ 0.55818576  0.7659338  -1.5065082  ...  1.544514    2.3999965
    1.2609208 ]
  [ 0.7113264   0.180385   -0.42705274 ...  0.4240638   2.4677043
    2.0552928 ]
  [ 0.91591203  0.34180903 -1.0722297  ...  0.9968933   2.822958
    2.5977676 ]
  [ 0.55982953  0.08902823 -0.49637258 ...  1.6795886   2.3831027
    1.269558  ]
  [ 1.100009    0.22682343  0.07365907 ...  1.1378539   2.7020812
    1.6387043 ]]

 [[ 0.7001849   1.4314605  -0.90819865 ...  0.56616014  1.197535
    2.247654  ]
  [ 0.6184092   1.5629302  -0.85190463 ...  0.9729141   2.1737888
    1.0227102 ]
  [ 1.107866    1.4481907  -0.60490626 ...  0.11705832  1.7010908
    1.5384034 ]
  [ 1.0612534   1.7052234  -0.94907767 ...  1.3872211   1.9995008
    1.2116961 ]
  [ 0.4205389   1.2733024  -0.60228366 ...  0.90708643  1.1303109
    1.4997379 ]]

 [[-0.30979607  0.09501119 -2.1249964  ...  1.8940775   1.7723631
    1.9347492 ]
  [ 0.3617029   0.33774242 -0.8522819  ...  2.098174    2.254145
    2.1955688 ]
  [ 

Running this code produces an output of shape, `(batch size, sequence length, model dimensionality)`. Note that you will likely see a different output due to the random initialization of the input sequence, and the parameter values of the Dense layers. 

<a id='sect4'></a>
## <font color='darkblue'>Further Reading</font>
This section provides more resources on the topic if you are looking to go deeper.
* **Books**
    * [Advanced Deep Learning with Python, 2019.](https://www.amazon.com/Advanced-Deep-Learning-Python-next-generation/dp/178995617X)
    * [Transformers for Natural Language Processing, 2021. ](https://www.amazon.com/Transformers-Natural-Language-Processing-architectures/dp/1800565798)
* **Papers**
    * [Attention Is All You Need, 2017.](https://arxiv.org/abs/1706.03762)